In [1]:
import re
import csv
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
rev = pd.read_csv("/home/ec2-user/SageMaker/bucket/wiki_trust/modeling/rev_id_list.txt",sep='\t')
del rev['Unnamed: 0'] 
rev.head()

,username,rev_id
0,$targlass,821350201
1,'lllllllllllllllllllllllllllllll',831073982
2,'lllllllllllllllllllllllllllllll',831059743
3,'lllllllllllllllllllllllllllllll',831059095
4,(127.0.0.1),842665329


In [3]:
rev.shape

(319512, 2)

### Testing for 1 user rev

In [4]:
user = '821350201'

In [5]:
new_url = 'https://ores.wikimedia.org/scores/enwiki/?models=damaging|goodfaith&revids='+user
new_url

'https://ores.wikimedia.org/scores/enwiki/?models=damaging|goodfaith&revids=821350201'

In [6]:
response = requests.get(new_url)
model_val = json.loads(response.text)
model_val

{'821350201': {'damaging': {'prediction': False,
   'probability': {'false': 0.9699778892213984, 'true': 0.030022110778601598}},
  'goodfaith': {'prediction': True,
   'probability': {'false': 0.01662229525825354, 'true': 0.9833777047417465}}}}

In [7]:
type(model_val)

dict

In [8]:
model_val[user]['damaging']['probability']['true']

0.030022110778601598

In [9]:
model_val[user]['goodfaith']['probability']['true']

0.9833777047417465

### Iterating for all user revisions

In [10]:
rev = rev.iloc[:50000,]
rev

,username,rev_id
0,$targlass,821350201
1,'lllllllllllllllllllllllllllllll',831073982
2,'lllllllllllllllllllllllllllllll',831059743
3,'lllllllllllllllllllllllllllllll',831059095
4,(127.0.0.1),842665329
5,(127.0.0.1),842663486
6,(127.0.0.1),842660787
7,(127.0.0.1),842486556
8,(127.0.0.1),481089675
9,(127.0.0.1),481076069


In [11]:
rev['damaging'] = ''
rev['goodFaith'] = ''
rev.head()

,username,rev_id,damaging,goodFaith
0,$targlass,821350201,,
1,'lllllllllllllllllllllllllllllll',831073982,,
2,'lllllllllllllllllllllllllllllll',831059743,,
3,'lllllllllllllllllllllllllllllll',831059095,,
4,(127.0.0.1),842665329,,


In [12]:
ex_list1 = []
ex_list2 = []
ex_list3 = []

In [13]:
start = time.time()
for index,row in rev.iterrows():
    revision = str(row['rev_id'])
    new_url = 'https://ores.wikimedia.org/scores/enwiki/?models=damaging|goodfaith&revids='+revision
    dam = 0.0
    gf = 0.0
    try:
        response = requests.get(new_url)
        model_val = json.loads(response.text)
    except:
        ex_list1.append(revision)
        
    try:
        dam = model_val[revision]['damaging']['probability']['true']
    except:
        ex_list2.append(revision)
        dam = 1.0
        
    try:
        gf = model_val[revision]['goodfaith']['probability']['true']
    except:
        ex_list3.append(revision)
        
    rev.at[index,'damaging'] = dam
    rev.at[index,'goodFaith'] = gf
end = time.time()    

In [14]:
print(end - start)

25861.693974494934


In [15]:
rev

,username,rev_id,damaging,goodFaith
0,$targlass,821350201,0.0300221,0.983378
1,'lllllllllllllllllllllllllllllll',831073982,0.0250485,0.981712
2,'lllllllllllllllllllllllllllllll',831059743,0.075616,0.971056
3,'lllllllllllllllllllllllllllllll',831059095,0.0944695,0.956036
4,(127.0.0.1),842665329,0.0242315,0.983864
5,(127.0.0.1),842663486,0.00552805,0.995399
6,(127.0.0.1),842660787,0.00746919,0.993269
7,(127.0.0.1),842486556,0.00657261,0.994693
8,(127.0.0.1),481089675,0.0175153,0.983735
9,(127.0.0.1),481076069,0.0150771,0.987699


In [16]:
print(len(ex_list1))
ex_list1

0


[]

In [17]:
print(len(ex_list2))
ex_list2

58


['714603844',
 '440601915',
 '704763278',
 '854986436',
 '610796584',
 '469414394',
 '847937302',
 '790494753',
 '790494694',
 '790494676',
 '790494643',
 '790494610',
 '790494519',
 '790494487',
 '790494328',
 '790494299',
 '790494259',
 '790494225',
 '790494194',
 '168719494',
 '669017753',
 '781391523',
 '789235322',
 '848080338',
 '844152387',
 '824482298',
 '768988056',
 '803517253',
 '781124042',
 '781123547',
 '822822408',
 '822838810',
 '822838803',
 '822838652',
 '822838640',
 '822838630',
 '822838622',
 '822838277',
 '822837875',
 '822837493',
 '837296136',
 '809593674',
 '809593597',
 '790466285',
 '790444704',
 '790432922',
 '790404858',
 '790339040',
 '790324943',
 '841103701',
 '776470101',
 '790810327',
 '804732126',
 '846686729',
 '809336941',
 '788058253',
 '848247271',
 '851904119']

In [18]:
print(len(ex_list3))
ex_list3

58


['714603844',
 '440601915',
 '704763278',
 '854986436',
 '610796584',
 '469414394',
 '847937302',
 '790494753',
 '790494694',
 '790494676',
 '790494643',
 '790494610',
 '790494519',
 '790494487',
 '790494328',
 '790494299',
 '790494259',
 '790494225',
 '790494194',
 '168719494',
 '669017753',
 '781391523',
 '789235322',
 '848080338',
 '844152387',
 '824482298',
 '768988056',
 '803517253',
 '781124042',
 '781123547',
 '822822408',
 '822838810',
 '822838803',
 '822838652',
 '822838640',
 '822838630',
 '822838622',
 '822838277',
 '822837875',
 '822837493',
 '837296136',
 '809593674',
 '809593597',
 '790466285',
 '790444704',
 '790432922',
 '790404858',
 '790339040',
 '790324943',
 '841103701',
 '776470101',
 '790810327',
 '804732126',
 '846686729',
 '809336941',
 '788058253',
 '848247271',
 '851904119']

In [19]:
rev.to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/ores_rev_1.csv')

In [20]:
pd.DataFrame(ex_list3).to_csv('/home/ec2-user/SageMaker/bucket/wiki_trust/revisions_data/csvs_stored/ores_rev_1_exc.csv')